# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [2]:
# import libraries
import pyspark as spark
from pyspark.sql import SparkSession
import zipfile
import numpy as np
import plotly.graph_objects as go

# unzip compressed data file
zip_ref = zipfile.ZipFile('mini_sparkify_event_data.json.zip', 'r')
zip_ref.extractall()
zip_ref.close()

In [3]:
# create a Spark session
spark = SparkSession.builder \
        .appName("Sparkify") \
        .getOrCreate()


# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [4]:
# load data set
data = spark.read.json("mini_sparkify_event_data.json")
data.show()
data.columns
data.count()

+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+---------------+-------------+---------+--------------------+------+-------------+--------------------+------+
|              artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|           page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+---------------+-------------+---------+--------------------+------+-------------+--------------------+------+
|      Martha Tilston|Logged In|    Colin|     M|           50| Freeman|277.89016| paid|     Bakersfield, CA|   PUT|       NextSong|1538173362000|       29|           Rockpools|   200|1538352117000|Mozilla/5.0 (Wind...|    30|
|    Five Iron Frenzy|Logged In|    Micah|     M|           79|    Long|236.09424| free|Bost

286500

In [19]:
data.createOrReplaceTempView("data")

In [20]:
# clean data
# Try to figure out which columns have missing values, are NULL
#for column in in data.columns

print('Check if Null')
for column in data.columns:
    print(column,spark.sql(f"SELECT * FROM data WHERE {column} IS NULL").count())
print()
print('Check if empty string')
for column in data.columns:
    print(column,spark.sql(f"SELECT * FROM data WHERE {column} = ''").count())

Check if Null
artist 58392
auth 0
firstName 8346
gender 8346
itemInSession 0
lastName 8346
length 58392
level 0
location 8346
method 0
page 0
registration 8346
sessionId 0
song 58392
status 0
ts 0
userAgent 8346
userId 0

Check if empty string
artist 0
auth 0
firstName 0
gender 0
itemInSession 0
lastName 0
length 0
level 0
location 0
method 0
page 0
registration 0
sessionId 0
song 0
status 0
ts 0
userAgent 0
userId 8346


- No missing values for auth, itemInSession, level, method, page, sessionID, status, ts, userID
- but 8346 empty string userIds
We are not going to consider empty string userIds therefore we create a new data frame

In [21]:
data_cleaned = spark.sql(f"SELECT * FROM data WHERE userId != ''")
data_cleaned.createOrReplaceTempView("data_cleaned")

In [22]:
# Number of unique items in each row
for column in data_cleaned.columns:
    print(column,spark.sql(f"SELECT COUNT(DISTINCT({column})) FROM data_cleaned").collect()[0][0])

artist 17655
auth 2
firstName 189
gender 2
itemInSession 1311
lastName 173
length 14865
level 2
location 114
method 2
page 19
registration 225
sessionId 2312
song 58480
status 3
ts 269770
userAgent 56
userId 225


- So 225 unique users, that we have to make predictions
- Lets make a plot 

In [23]:
# Comparison of number of interactions
paid_user_interactions = np.array(data_cleaned[data_cleaned['level']=='paid'].groupBy('userId').count().orderBy('count', ascending=False).collect()).astype(int)
free_user_interactions = np.array(data_cleaned[data_cleaned['level']=='free'].groupBy('userId').count().orderBy('count', ascending=False).collect()).astype(int)

# Comparison number of paied and free users
fig = go.Figure()
fig.add_trace(go.Pie(labels=['free','paid'], values = [free_user_interactions.shape[0], paid_user_interactions.shape[0]]))
fig.update_layout(title_text="Fraction of free and paying users",
                  template='plotly_white')
fig.show()
# Comparison of number of interactions
fig = go.Figure()
fig.add_trace(go.Histogram(x=free_user_interactions[:,1], name = 'free'))
fig.add_trace(go.Histogram(x=paid_user_interactions[:,1], name = 'paid'))
fig.update_xaxes(title_text='Number of interactions')
fig.update_yaxes(title_text='Number of users')
fig.update_layout(title_text="Comparison interactions free and paying users",
                  template='plotly_white')
fig.show()

In [24]:
# Statistics about the number of interactions per session
free_interactions_per_session = np.array(data_cleaned[data_cleaned['level'] == 'free'].groupBy('sessionId').count().collect()).astype(int)
paid_interactions_per_session = np.array(data_cleaned[data_cleaned['level'] == 'paid'].groupBy('sessionId').count().collect()).astype(int)
fig = go.Figure()
fig.add_trace(go.Box(y=free_interactions_per_session[:,1], name = 'Free'))
fig.add_trace(go.Box(y=paid_interactions_per_session[:,1], name = 'Paid'))
fig.update_yaxes(title_text='Number of interactions per session')
fig.update_layout(title_text="Boxplot number of interactions per session",
                  template='plotly_white')
fig.show()

In [26]:
# Count of the interactions with the different pages
free_count_of_pages = np.array(data_cleaned[data_cleaned['level'] == 'free'].groupBy('page').count().orderBy('count', ascending=False).collect())
paid_count_of_pages = np.array(data_cleaned[data_cleaned['level'] == 'paid'].groupBy('page').count().orderBy('count', ascending=False).collect())
fig = go.Figure()
fig.add_trace(go.Bar(x=free_count_of_pages[:,1].astype(float), y=free_count_of_pages[:,0], orientation='h', name='free'))
fig.add_trace(go.Bar(x=paid_count_of_pages[:,1].astype(float), y=paid_count_of_pages[:,0], orientation='h', name='paid'))
fig.update_layout(title_text="Number of page interactions",
                  height=600,
                  template='plotly_white')
fig.show()

In [27]:
# Count of the interactions with the different pages for unique users per page
import pyspark.sql.functions as func
free_count_of_pages = np.array(data_cleaned[data_cleaned['level'] == 'free'].groupby('page').agg(func.countDistinct('userID')).collect())
paid_count_of_pages = np.array(data_cleaned[data_cleaned['level'] == 'paid'].groupby('page').agg(func.countDistinct('userID')).collect())


fig = go.Figure()
fig.add_trace(go.Bar(x=free_count_of_pages[:,1].astype(float), y=free_count_of_pages[:,0], orientation='h', name='unique free users'))
fig.add_trace(go.Bar(x=paid_count_of_pages[:,1].astype(float), y=paid_count_of_pages[:,0], orientation='h', name='unqiue paid users'))
fig.update_layout(title_text="Number of page interactions for unqiue users",
                  height=600,
                  template='plotly_white')
fig.show()


In [217]:
data_cleaned.show()

+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+---------------+-------------+---------+--------------------+------+-------------+--------------------+------+
|              artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|           page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+---------------+-------------+---------+--------------------+------+-------------+--------------------+------+
|      Martha Tilston|Logged In|    Colin|     M|           50| Freeman|277.89016| paid|     Bakersfield, CA|   PUT|       NextSong|1538173362000|       29|           Rockpools|   200|1538352117000|Mozilla/5.0 (Wind...|    30|
|    Five Iron Frenzy|Logged In|    Micah|     M|           79|    Long|236.09424| free|Bost

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [30]:
data_cleaned.columns

['artist',
 'auth',
 'firstName',
 'gender',
 'itemInSession',
 'lastName',
 'length',
 'level',
 'location',
 'method',
 'page',
 'registration',
 'sessionId',
 'song',
 'status',
 'ts',
 'userAgent',
 'userId']

In [29]:
# Add Churn column
def valueToCategory(value):
   if   value == 'Cancellation Confirmation': return 1
   else : return 0
from pyspark.sql.functions import udf
from pyspark.sql.types import *

udfValueToCategory = udf(valueToCategory, StringType())
data_feat_eng = data_cleaned.withColumn("Churn", udfValueToCategory('page'))
data_feat_eng.groupBy('Churn').count().show()

+-----+------+
|Churn| count|
+-----+------+
|    0|278102|
|    1|    52|
+-----+------+



# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.